In [24]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as rq
import json
import time
import random

# Login to ResearchGate

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.window import WindowTypes

driver = webdriver.Firefox()
driver.get("https://www.researchgate.net/login")

password = driver.find_element(By.ID, "input-password")
username = driver.find_element(By.ID, "input-login")

password.send_keys("kcaeDHT8uE5wYMe")
username.send_keys("matanki@my.hit.ac.il")
password.send_keys(Keys.RETURN)

## Columns of the data-frame

In [26]:
fullname = []
gender = []
position = []
nation = []
institute = []
shanghai_ranking = []
depratment = []
disciplines = []
publications = []
citations = []
recommendations = []
top_cited_research = []
# top_cited_research_journal = []
interest_score = []
reads = []
h_index = []
URL = []

## Lists of institutes around the world
* *instLst_1* - Israeli universities and collages
* *instLst_2* - Ivy League universities - Eight private research universities in the Northeastern United States.
* *instLst_3* - Top internationally universities
* *instLst_4* - internationally universities
* *instLst_5* - Latin america universities

In [27]:
instLst_1 = ["Holon-Institute-of-Technology", "Tel Aviv University",
           "Hebrew University of Jerusalem", "Bar Ilan University",
           "Technion-Israel Institute of Technology", "The-Open-University-of-Israel",
           "Ben-Gurion-University-of-the-Negev", "Ariel-University",
           "University-of-Haifa", "Weizmann-Institute-of-Science",
           "Academic College of Tel Aviv-Yafo", "Shamoon-College-of-Engineering"]

instLst_2 = ["Brown University", "Harvard University",
           "Yale-University", "University of Pennsylvania",
           "Columbia-University", "Geisel-School-of-Medicine",
           "Princeton University", "Cornell University"]

instLst_3 = ["Stanford-University", "Massachusetts-Institute-of-Technology",
            "University of Cambridge", "University-of-California-Berkeley",
            "University-of-Oxford", "Universite_Paris-Saclay",
            "Ecole-Polytechnique", "ETH-Zurich", "University-of-Toronto",
            "The University of Tokyo", "Tsinghua University"]

instLst_4 = ["Ege-University", "Hallym-University",
            "University of Iceland", "University of Delhi",
            "University of Cyprus", "Umea-University",
            "Tampere-University", "Tehran University of Medical Sciences",
            "University-of-Malaya", "Universita degli Studi di Milano-Bicocca",
            "University-of-Granada"]

instLst_5 = ["Pontificia Universidad Católica de Chile", "Universidade de São Paulo",
            "Instituto Tecnológico y de Estudios Superiores de Monterrey",
            "Universidad de los Andes", "Universidade Estadual de Campinas",
            "Universidad Nacional Autónoma de Mexico", "Universidad de Chile",
            "Universidad de Buenos Aires", "Universidade Federal do Rio de Janeiro",
            "Universidad Nacional de Colombia", "UNESP", "Universidad de Concepción",
            "Universidad de Santiago de Chile (USACH)", "Universidad de Antioquia",
            "Pontificia Universidade Católica do Rio de Janeiro", "Pontificia Universidad Javeriana",
            "Universidade Federal de Minas Gerais", "Pontificia Universidad Católica del Peru",
            "Rica Universidad de Costa Rica", "Universidade Federal Rio Grande do Sul"]

## Shanghai ranking & Nations dict.

In [29]:
# dictionary
inst_rank = {
    "Holon-Institute-of-Technology":801,
    "Tel Aviv University":151,
    "Hebrew University of Jerusalem":77,
    "Bar Ilan University":301,
    "Technion-Israel Institute of Technology":83,
    "The-Open-University-of-Israel":201,
    "Ben-Gurion-University-of-the-Negev":401,
    "Ariel-University":801,
    "University-of-Haifa":501,
    "Weizmann-Institute-of-Science":83,
    "Academic College of Tel Aviv-Yafo":801,
    "Shamoon-College-of-Engineering":801,
    "Brown University":47,
    "Harvard University":1,
    "Yale-University":11,
    "University of Pennsylvania":15,
    "Columbia-University":8,
    "Geisel-School-of-Medicine":301,
    "Princeton University":6,
    "Cornell University":12,
    "Stanford-University":2,
    "Massachusetts-Institute-of-Technology":3,
    "University of Cambridge":4,
    "University-of-California-Berkeley":5,
    "University-of-Oxford":7,
    "Universite_Paris-Saclay":16,
    "Ecole-Polytechnique":301,
    "ETH-Zurich":20,
    "University-of-Toronto":22,
    "The University of Tokyo":24,
    "Tsinghua University":26,
    "Ege-University":901,
    "Hallym-University":901,
    "University of Iceland":601,
    "University of Delhi":601,
    "University of Cyprus":501,
    "Umea-University":501,
    "Tampere-University":401,
    "Tehran University of Medical Sciences":401,
    "University-of-Malaya":301,
    "Universita degli Studi di Milano-Bicocca":301,
    "University-of-Granada":201,
    'Pontificia_Universidad_Catolica_de_Chile' : 500 , 
    'University-of-Sao-Paulo':101 ,
    'Los-Andes-University-Colombia':101 ,
    'University_of_Campinas':150 ,
    'Universidad_Nacional_Autonoma_de_Mexico' : 405 ,
    'University-of-Chile': 401 , 
    'Universidad_de_Buenos_Aires' :67  , 
    'Federal-University-of-Rio-de-Janeiro2' : 401 ,
    'National_University_of_Colombia' : 801,
    'Sao_Paulo_State_University' : 801 ,
    'Universidad_de_Concepcion':801 ,
    'University-of-Santiago-Chile' : 401
}
    

inst_nation = {
    "Holon-Institute-of-Technology":"Israel",
    "Tel Aviv University":"Israel",
    "Hebrew University of Jerusalem":"Israel",
    "Bar Ilan University":"Israel",
    "Technion-Israel Institute of Technology":"Israel",
    "The-Open-University-of-Israel":"Israel",
    "Ben-Gurion-University-of-the-Negev":"Israel",
    "Ariel-University":"Israel",
    "University-of-Haifa":"Israel",
    "Weizmann-Institute-of-Science":"Israel",
    "Academic College of Tel Aviv-Yafo":"Israel",
    "Shamoon-College-of-Engineering":"Israel",
    "Brown University":"United States",
    "Harvard University":"United States",
    "Yale-University":"United States",
    "University of Pennsylvania":"United States",
    "Columbia-University":"United States",
    "Geisel-School-of-Medicine":"United States",
    "Princeton University":"United States",
    "Cornell University":"United States",
    "Stanford-University":"United States",
    "Massachusetts-Institute-of-Technology":"United States",
    "University of Cambridge":"Great Britain",
    "University-of-California-Berkeley":"United States",
    "University-of-Oxford":"Great Britain",
    "Universite_Paris-Saclay":"France",
    "Ecole-Polytechnique":"France",
    "ETH-Zurich":"Switzerland",
    "University-of-Toronto":"Canada",
    "The University of Tokyo":"Japan",
    "Tsinghua University":"Chaina",
    "Ege-University":"Turkey",
    "Hallym-University":"South Korea",
    "University of Iceland":"Iceland",
    "University of Delhi":"India",
    "University of Cyprus":"Cyprus",
    "Umea-University":"Sweden",
    "Tampere-University":"Finland",
    "Tehran University of Medical Sciences":"Iran",
    "University-of-Malaya":"Malaysia",
    "Universita degli Studi di Milano-Bicocca":"Italy",
    "University-of-Granada":"Spain"
    'Pontificia_Universidad_Catolica_de_Chile' : "Chile" , 
    'University-of-Sao-Paulo' : "Brazil" ,
    'Los-Andes-University-Colombia':"Colombia" ,
    'University_of_Campinas': "Brazil" ,
    'Universidad_Nacional_Autonoma_de_Mexico' : "Mexico" ,
    'University-of-Chile': "Chile" , 
    'Universidad_de_Buenos_Aires' : "Argentina"  , 
    'Federal-University-of-Rio-de-Janeiro2' : "Brazil",
    'National_University_of_Colombia' : "Colombia",
    'Sao_Paulo_State_University' : "Brazil",
    'Universidad_de_Concepcion':"Chile" ,
    'University-of-Santiago-Chile' : "Chile"
}

## Determine the gender by name using API function

In [30]:
def genderFromName(string):
    name = string.split()
    name = name[0]
#     print(name) # taking only firstname
    url = "https://api.genderize.io?name="+name
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0'}
    file = rq.get(url, headers=headers)
    if str(file.content).find("female") == -1: #TODO: maybe a better way in time consuming.
        return "male"
    else:
        return "female"

## Crawling and gathering information from ResearchGate function

In [31]:
def crawler(instLst, NumOfSientistPerInstatue):
    for x in instLst:
        driver.get("https://www.researchgate.net/institution/"+x+"/members")
        content = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[3]/div/div/nav/div[7]/a/span')
        index = bs(content.get_property("innerHTML"), "html.parser").get_text()
    #     print(index) # Number of member pages for each institue

        limit = min(int(index) + 1, NumOfSientistPerInstatue + 1) # we don't mind more than 100*10 researchers per institue

        for i in range(1, limit): #int(index) + 1

            driver.get("https://www.researchgate.net/institution/"+x+"/members/"+str(i))
            p = driver.current_window_handle

            for j in range(1, 11):

                try:
                    name = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[1]/div/div/h5/a")
                    url = name.get_attribute('href')
                    name = bs(name.get_property("innerHTML"), "html.parser").get_text()
                    print(url)
                    print(name)
                except:
                    break #no more people to show

                try:
                    Gender = genderFromName(name)
                except:
                    Gender = "Nan"

                print (Gender)
                print(x)
                print(inst_rank[x])
                try:
                    Department = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[2]/div/div[2]/ul/li/span")
                    Department = bs(Department.get_property("innerHTML"), "html.parser").get_text()
                    print(Department)
                except:
                    Department = "Nan"
                    print("Nan")

                try: 
                    Disciplines = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[3]/div/div[2]")
                    Disciplines = bs(Disciplines.get_property("innerHTML"), "html.parser").get_text("|", strip=True)
    #                 A = [text for text in disciplines.stripped_strings] # getting list of disciplines instead of on string
                    print(Disciplines)
                except:
                    Disciplines = "Nan"
                    print("Nan")

                # all data on the members' page was collected    
                # now we want to reach the web page of the sientist

                driver.switch_to.new_window(WindowTypes.TAB)
                driver.get(url+'/stats')

                try: 
                    Research_Interest_Score = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[1]/div/div/div/div[1]/div")
                    Research_Interest_Score = bs(Research_Interest_Score.get_property("innerHTML"), "html.parser").get_text()
                    if Research_Interest_Score == '- -':
                        Research_Interest_Score = "0";
                    print(Research_Interest_Score)
                except:
                    Research_Interest_Score = "0"
                    print("Nan")

                try: 
                    Reads = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[2]/div/div/div/div[1]/div")
                    Reads = bs(Reads.get_property("innerHTML"), "html.parser").get_text()
                    print(Reads)
                except:
                    Reads = "0"
                    print("Nan")

                try: 
                    Citations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[3]/div/div/div/div[1]/div")
                    Citations = bs(Citations.get_property("innerHTML"), "html.parser").get_text()
                    print(Citations)
                except:
                    Citations = "0"
                    print("Nan")

                try: 
                    Recommendations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[4]/div/div/div/div[1]/div")
                    Recommendations = bs(Recommendations.get_property("innerHTML"), "html.parser").get_text()
                    print(Recommendations)
                except:
                    Recommendations = "0"
                    print("Nan")

                try: 
                    hindex = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[1]/ul/li[1]/div/div/div/div[1]/div")
                    hindex = bs(hindex.get_property("innerHTML"), "html.parser").get_text()
                    print(hindex)
                except:
                    hindex = "0"
                    print("Nan")

                try: 
                    Top_cited_research = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[2]/div/div/div/div[2]/div/a/strong")
                    Top_cited_research = bs(Top_cited_research.get_property("innerHTML"), "html.parser").get_text()
                    print(Top_cited_research)
                except:
                    Top_cited_research = "Nan"
                    print("Nan")

                try: 
                    Position = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div[3]/ul/li[1]")
                    Position = bs(Position.get_property("innerHTML"), "html.parser").get_text()
                    word = Position.split()[0]
                    if word == x.split()[0] or word == x.split('-')[0]:
                        Position = "Nan"
                    print(Position)
                except:
                    Position = "Nan"
                    print("Nan")

                try: 
                    no_publications = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div/div[1]/div[2]/div/div/div/div[1]/div/nav/div/div[1]/a[2]/div/div')
                    no_publications = bs(no_publications.get_property("innerHTML"), "html.parser").get_text()

                    # reciving the number of publications

                    no_publications = no_publications.split()[1].strip('()')
                    print(no_publications)

                except:
                    no_publications = "0"
                    print("0")


                # now we've done scraping all relevent data on this sientinst
                # we close the tab and return to previous tab

                time.sleep(random.uniform(1.9, 2.4)) # In order to react less suspicious
                driver.close()
                driver.switch_to.window(p)

                print("\n")

                fullname.append(name)
                gender.append(Gender)
                position.append(Position)
                nation.append(inst_nation[x])
                institute.append(x)
                shanghai_ranking.append(inst_rank[x])
                depratment.append(Department)
                disciplines.append(Disciplines)
                publications.append(float(no_publications.replace(",", "").replace(".", "")))
                citations.append(float(Citations.replace(",", "").replace(".", "")))
                recommendations.append(float(Recommendations.replace(",", "").replace(".", "")))
                top_cited_research.append(Top_cited_research)
                # top_cited_research_journal +=
                interest_score.append(float(Research_Interest_Score.replace(",", "").replace(".", "")))
                reads.append(float(Reads.replace(",", "").replace(".", "")))
                h_index.append(float(hindex.replace(",", "").replace(".", "")))
                URL.append(url)


In [36]:
crawler(instLst_4, 100)


https://www.researchgate.net/profile/Anita-Tewari
Anita Tewari
male
Geisel-School-of-Medicine
301
Department of Microbiology and Immunology
Immunohistochemistry|PCR|Gel Electrophoresis
463.1
42,005
457
73
9
Bacillus cereus food poisoning: international and Indian perspective
Doctor of Veterinary Medicine
37


https://www.researchgate.net/profile/Geoffrey-Eglinton
Geoffrey Eglinton
male
Geisel-School-of-Medicine
301
Department of Earth Sciences (EARS)
Nan
17,909
36,131
38,270
42
97
Leaf Epicuticular Waxes
Professor (Associate) at Dartmouth College
522


https://www.researchgate.net/profile/Xun-Shi-2
Xun Shi
male
Geisel-School-of-Medicine
301
Department of Geography
Geography|Geostatistics|Spatial Analysis
2,285
27,958
3,981
124
36
Geographic Access to Cancer Care in the U.S
PhD
117


https://www.researchgate.net/profile/Margaret-Ackerman
Margaret Ackerman
male
Geisel-School-of-Medicine
301
Thayer School of Engineering
Antibodies
2,329
23,909
5,975
77
42
Polyfunctional Fc-Effector Profil

male
Geisel-School-of-Medicine
301
Medical Genetics
Nan
426.9
3,835
929
66
14
Large-scale association analysis identifies new lung cancer susceptibility loci and heterogeneity in genetic susceptibility across histological subtypes
MD PhD
42


https://www.researchgate.net/profile/Haiyi-Xie
Haiyi Xie
male
Geisel-School-of-Medicine
301
Department of Biomedical Data Science, and Department of Community and Family Medicine
Nan
5,393
71,960
10,391
84
57
Assertive Community Treatment for Patients with Co-Occurring Severe Mental Illness and Substance Use Disorder
Ph.D.
305


https://www.researchgate.net/profile/Paul-Whalen
Paul J Whalen
male
Geisel-School-of-Medicine
301
Department of Psychological and Brain Sciences
fMRI
11,895
35,162
24,253
18
58
Davis M, Whalen PJ. The amygdala: vigilance and emotion. Mol Psychiatr 6: 13-34
PhD, Professor
111


https://www.researchgate.net/profile/David-Leib
David A Leib
male
Geisel-School-of-Medicine
301
Department of Microbiology and Immunology
Virus|Neur

male
Geisel-School-of-Medicine
301
Biomedical Data Science and Psychiatry
digital phenotyping|digital interventions|Machine Learning
1,285
30,280
1,789
70
23
Anxiety and Depression as Bidirectional Risk Factors for One Another: A Meta-Analysis of Longitudinal Studies
Ph.D. in Clinical Psychology
125


https://www.researchgate.net/profile/Wenyan-Zhao-3
Wenyan Zhao
male
Geisel-School-of-Medicine
301
Geisel School of Medicine
Statistical Analysis|Statistical Design|SAS
1,689
4,782
3,484
5
33
Surgical Compared with Nonoperative Treatment for Lumbar Degenerative Spondylolisthesis Four-Year Results in the Spine Patient Outcomes Research Trial (SPORT) Randomized and Observational Cohorts
Ph.D.
77


https://www.researchgate.net/profile/Dana-Williams-2
Dana P. Williams
male
Geisel-School-of-Medicine
301
Department of Mathematics
Mathematical Analysis|Topology|Functional Analysis
1,284
5,726
2,987
29
23
Morita equivalence and continuous-trace C * -algebras
PhD
101


https://www.researchgate.net/

2,542
31,579
4,250
75
32
A Review of Design Principles for Community-Based Natural Resource Management
PhD
66


https://www.researchgate.net/profile/Jifeng-Liu-6
Jifeng Liu
male
Geisel-School-of-Medicine
301
Thayer School of Engineering
Materials Engineering|Material Characterization|Photonics
4,731
38,638
9,709
41
43
High-performance Ge-on-Si Photodetectors
Ph.D.
230


https://www.researchgate.net/profile/Solomon-Diamond
Solomon Gilbert Diamond
male
Geisel-School-of-Medicine
301
Thayer School of Engineering
Functional Neuroimaging
1,651
18,456
3,210
10
21
HomER: A review of time-series analysis methods for near-infrared spectroscopy of the brain
PhD
65


https://www.researchgate.net/profile/Geoffrey-Parker-2
Geoffrey Parker
male
Geisel-School-of-Medicine
301
Thayer School of Engineering
E-Business|Information System Management|Economics of Regulation
4,258
111,439
6,421
41
29
Strategies for two-sided markets
PhD
84


https://www.researchgate.net/profile/Nathaniel-Dominy
Nathaniel J Do

male
Geisel-School-of-Medicine
301
Health Systems
Nan
1,017
22,370
2,053
3
22
Microsystems In Health Care: Part 1. Learning From High-Performing Front-Line Clinical Units
PhD, MS, BSN
30


https://www.researchgate.net/profile/Brenden-Epps
Brenden Epps
male
Geisel-School-of-Medicine
301
Thayer School of Engineering
Aerodynamics|Numerical Modeling|Computational Fluid Dynamics
1,050
37,777
1,375
20
17
Water Entry of Projectiles
Ph.D. MIT Mechanical Engineering (fluid dynamics)
53


https://www.researchgate.net/profile/Michael-Daily-2
Michael F Daily
male
Geisel-School-of-Medicine
301
Department of Surgery
Laparoscopic Surgery|Minimally Invasive Surgery|Hepatocellular Carcinoma
563.7
2,093
1,171
14
17
Liver Transplantation for the Treatment of Liver Metastases From Neuroendocrine Tumors An Analysis of the UNOS Database
MD, MS, FACS
52


https://www.researchgate.net/profile/Sarah-Munro-5
Sarah Munro
male
Geisel-School-of-Medicine
301
The Dartmouth Institute for Health Policy and Clinical Pr

...



https://www.researchgate.net/profile/John-Parker-28
John S L Parker
male
Cornell University
12
Baker Institute for Animal Health
Infection|Virus|Proteins
961.8
5,093
2,253
3
24
Canine and Feline Parvoviruses Can Use Human or Feline Transferrin Receptors To Bind, Enter, and Infect Cells
DVM, PhD
29


https://www.researchgate.net/profile/Ethan-Degner
Ethan Degner
male
Cornell University
12
Department of Entomology
Nan
156.0
4,349
236
12
7
Mating-Induced Transcriptome Changes in the Reproductive Tract of Female Aedes aegypti
PhD Student at Cornell University
31


https://www.researchgate.net/profile/Ramin-Pajoum-Shariati
Ramin Pajoum shariati
male
Cornell University
12
Biocompatibility
Nan
385.9
8,046
665
20
16
The Effect of Hydraulic Retention Time on the Performance and Fouling Characteristics of Membrane Sequencing Batch Reactors Used for the Treatment of Synthetic Petroleum Refinery Wastewater
Doctor of Philosophy
29


https://www.researchgate.net/profile/Robert-Munroe
Robert J Munr

male
Cornell University
12
Charles H. Dyson School of Applied Economics and Management
Agricultural Economics|Advertising
637.9
23,868
1,161
23
20
Consumer Valuation of Environmentally Friendly Production Practices in Wines, considering Asymmetric Information and Sensory Effects
PhD Applied Economics and Management
95


https://www.researchgate.net/profile/Louisa-Smieska
Louisa Smieska
male
Cornell University
12
Cornell High Energy Synchrotron Source
Organic Electronics|Thin Film Deposition|Scanning Probe Microscopy
207.9
6,921
201
21
9
Revealing the biography of a hidden medieval manuscript using synchrotron and conventional imaging techniques
PhD
29


https://www.researchgate.net/profile/Jared-Strait
Jared H. Strait
male
Cornell University
12
Department of Electrical and Computer Engineering
Plasmonics|Graphene|Femtosecond Lasers
1,051
7,204
2,060
20
11
Ultrafast Optical-Pump Terahertz-Probe Spectroscopy of the Carrier Relaxation and Recombination Dynamics in Epitaxial Graphene
Nan
2



https://www.researchgate.net/profile/Suresh-Vishwanath
Suresh Vishwanath
male
Cornell University
12
Microfabrication
Nan
496.2
6,259
932
1
14
Exfoliated multilayer MoTe2 field-effect transistors
Nan
29


https://www.researchgate.net/profile/Darryl-Seligman
Darryl Seligman
male
Cornell University
12
Department of Astronomy
Nan
198.9
2,176
486
10
11
The Feasibility and Benefits of In Situ Exploration of ‘Oumuamua-like Objects
Nan
49


https://www.researchgate.net/profile/Rocio-Gonzalez-6
Rocío González
male
Cornell University
12
Department of Biomedical Sciences
Cetaceans|Necropsy|Histopathology
375.9
7,844
551
33
12
Brucella ceti and Brucellosis in Cetaceans
DVM
27


https://www.researchgate.net/profile/Ranju-Ravindran-Santhakumari-Manoj
Ranju Ravindran Santhakumari Manoj
male
Cornell University
12
Dogs
Nan
245.6
3,797
223
147
8
Ticks and associated pathogens in camels (Camelus dromedarius) from Riyadh Province, Saudi Arabia
Post doctoral associate and resident at Cornell University
2

2,233
6,902
4,700
5
38
Elastic modulus and equilibrium swelling of poly(dimethylsiloxane) networks
PhD
138


https://www.researchgate.net/profile/Jafar-Jafarzadeh-2
Jafar Jafarzadeh
male
Cornell University
12
Department of Plant Breeding and Genetics
Plant Breeding|Plant Biotechnology|Molecular Markers
122.3
3,496
94
29
5
Breeding Value of Primary Synthetic Wheat Genotypes for Grain Yield
Ph.D. Plant Breeding, Wheat Breeder
27


https://www.researchgate.net/profile/Anastasia-Dalziell
Anastasia H. Dalziell
male
Cornell University
12
Department of Neurobiology and Behavior
Behavioral Ecology|Ecology and Evolution|Evolution
387.0
11,355
607
23
12
Song matching, overlapping, and switching in the banded wren: The sender's perspective
B.Mus./B.Sc, PhD
26


https://www.researchgate.net/profile/Mahdi-Hesari
Mahdi Hesari
male
Cornell University
12
Department of Chemistry and Chemical Biology
Synthesis|Cyclic Voltammetry|Organic Synthesis
749.2
5,681
1,770
28
22
Review—Electrogenerated Chemilumi

249.7
11,140
361
7
9
Florigenic And Antiflorigenic Signalling In Plants.
PhD (Warwick, 2010)
25




In [37]:
df = pd.DataFrame({"Full name":fullname, "Gender":gender, "Position":position,
                   "Nation":nation, "Institute":institute, "Shanghai_ranking":shanghai_ranking,
                   "Depratment":depratment, "Disciplines":disciplines, "Publications":publications,
                   "Citations":citations, "Recommendations":recommendations,
                   "Top cited research":top_cited_research, "Interest score":interest_score, 
                   "Reads":reads, "h index":h_index, "URL":URL})

df.to_csv('sientinst_data')

In [38]:
df

,Full name,Gender,Position,Nation,Institute,Shanghai_ranking,Depratment,Disciplines,Publications,Citations,Recommendations,Top cited research,Interest score,Reads,h index,URL
0,sheng zhang,male,Ph.D.,United States,Yale-University,11,Department of Psychiatry,Pattern Recognition|C/C++/C#|Functional Neuroi...,113.0,4426.0,35.0,Functional connectivity mapping of the human p...,1963.0,16095.0,35.0,https://www.researchgate.net/profile/Sheng-Zha...
1,Jon Arellano,male,PhD Neurobiology,United States,Yale-University,11,Department of Neurobiology,Neuron|Brain|Synapses,49.0,3302.0,22.0,Microstructure of the neocortex: Comparative a...,1628.0,12919.0,21.0,https://www.researchgate.net/profile/Jon_Arellano
2,Guangchuan Wang,male,PhD,United States,Yale-University,11,Department of Genetics,Calcium Phosphates|Materials Chemistry|Chemica...,50.0,1705.0,13.0,AAV-mediated direct in vivo CRISPR screen iden...,9154.0,14842.0,24.0,https://www.researchgate.net/profile/Guangchua...
3,Anna Rachel Gallagher,male,PhD,United States,Yale-University,11,Section of Nephrology,Fibrosis|Experimental Nephrology|Chronic Renal...,38.0,1681.0,14.0,A genetic interaction network of five genes fo...,7479.0,4143.0,19.0,https://www.researchgate.net/profile/Anna_Rach...
4,Brett Lindenbach,male,PhD,United States,Yale-University,11,Department of Microbial Pathogenesis,RNA|Molecular Cloning|PCR,117.0,15148.0,27.0,Virology: Complete replication of hepatitis C ...,7214.0,20280.0,41.0,https://www.researchgate.net/profile/Brett_Lin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Hamid Razifard,male,PhD,United States,Cornell University,12,Department of Plant Biology,Next Generation Sequencing|PCR|Gel Electrophor...,26.0,510.0,51.0,Major Impacts of Widespread Structural Variati...,3970.0,6562.0,7.0,https://www.researchgate.net/profile/Hamid-Raz...
5996,Samuel Schmidt,male,PhD,United States,Cornell University,12,Department of Chemical and Biomolecular Engine...,Biochemistry|Nanomedicine|Spectroscopy,25.0,1131.0,11.0,Readily Accessible Bicyclononynes for Bioortho...,6058.0,10662.0,12.0,https://www.researchgate.net/profile/Samuel-Sc...
5997,Clément Goubert,male,PhD in Evolutionary biology and genomics,United States,Cornell University,12,Department of Molecular Biology and Genetics,Behavioral Ecology|Entomology|Evolutionary Bio...,35.0,1321.0,75.0,RepeatModeler2 for automated genomic discovery...,7798.0,9756.0,14.0,https://www.researchgate.net/profile/Clement-G...
5998,Gloria Gioia,male,PhD,United States,Cornell University,12,Department of Population Medicine and Diagnost...,DNA Extraction|RNA|PCR,25.0,419.0,34.0,Highly sensitive multiplex PCR for simultaneou...,2706.0,5291.0,12.0,https://www.researchgate.net/profile/Gloria-Gioia
